# Overview

The following calculation computes the end-to-end measurement uncertainty of the high-side current-sensing circuit design proposed by this [application note](https://www.ti.com/lit/an/sboa310a/sboa310a.pdf) published by Texas Instruments. A more detailed discussion about this code and how it can be best applied to real-world engineering scenarios is available in this [blog post](https://www.osborneee.com/worstcase/).

In [1]:
from pprint import pprint as print

import numpy as np
from scipy.linalg import solve

from worstcase import derive, param, unit

## Define Input Parameters

In [2]:
# define the resistor uncertainties
R1 = param.bytol(nom=100 * unit.mohm, tol=0.01, rel=True, tag="R1")
R2 = param.bytol(nom=1.001 * unit.kohm, tol=0.01, rel=True, tag="R2")
R3 = param.bytol(nom=50.5 * unit.kohm, tol=0.01, rel=True, tag="R3")
R4 = param.bytol(nom=1.001 * unit.kohm, tol=0.01, rel=True, tag="R4")
R5 = param.bytol(nom=50.5 * unit.kohm, tol=0.01, rel=True, tag="R5")

# define the amplifier offset voltage
VOS = param.bytol(nom=0 * unit.mV, tol=150 * unit.uV, rel=False, tag="VOS")

print([R1, R2, R3, R4, R5, VOS])

[R1: 100 mΩ (nom), 99 mΩ (lb), 101 mΩ (ub),
 R2: 1.001 kΩ (nom), 991 Ω (lb), 1.011 kΩ (ub),
 R3: 50.5 kΩ (nom), 49.99 kΩ (lb), 51.01 kΩ (ub),
 R4: 1.001 kΩ (nom), 991 Ω (lb), 1.011 kΩ (ub),
 R5: 50.5 kΩ (nom), 49.99 kΩ (lb), 51.01 kΩ (ub),
 VOS: 0 mV (nom), -150 µV (lb), 150 µV (ub)]


## Define End-to-End Uncertainty Equations

In [3]:
# define the output voltage
@derive.byev(r1=R1, r2=R2, r3=R3, r4=R4, r5=R5, vos=VOS)
def VO(vbus, iload, r1, r2, r3, r4, r5, vos):
    vp = vbus * r3 / (r2 + r3)
    vn = vp + vos
    vo = vn - (vbus - r1 * iload - vn) * r5 / r4
    return vo

In [4]:
# define the end-to-end uncertainty
@derive.byev(r1=R1, r2=R2, r3=R3, r4=R4, r5=R5, vos=VOS)
def IUNC(r1, r2, r3, r4, r5, vos, vbus, iload):
    vo = VO(vbus, iload, r1, r2, r3, r4, r5, vos)
    return vo / VO(vbus, iload).nom * iload - iload

## Calculate End-to-End Uncertainty @ 36V, 1A

In [5]:
# calculate at 36V, 1A operating point
VOUT_1A = VO(vbus=36 * unit.V, iload=1 * unit.A, tag="VOUT_1A")
IUNC_1A = IUNC(vbus=36 * unit.V, iload=1 * unit.A, tag="IUNC_1A")

print([VOUT_1A, IUNC_1A])

[VOUT_1A: 5.045 V (nom), 3.647 V (lb), 6.387 V (ub),
 IUNC_1A: 0 A (nom), -277 mA (lb), 266 mA (ub)]


In [6]:
# perform sensitivity study at the 36V, 1A operating point
IUNC_1A_sensitivities = [
    IUNC_1A(tag="IUNC_1A-R1-sens").ss(R1),
    IUNC_1A(tag="IUNC_1A-VOS-sens").ss(VOS),
    IUNC_1A(tag="IUNC_1A-R2-thru-R5-sens").ss([R2, R3, R4, R5]),
]

print(IUNC_1A_sensitivities)

[IUNC_1A-R1-sens: 0 A (nom), -10 mA (lb), 10 mA (ub),
 IUNC_1A-VOS-sens: 0 A (nom), -1.53 mA (lb), 1.53 mA (ub),
 IUNC_1A-R2-thru-R5-sens: 0 A (nom), -265.3 mA (lb), 254.7 mA (ub)]


## Calculate End-to-End Uncertainty @ 36V, 50mA

In [7]:
# calculate at 36V, 50mA operating point
VOUT_50mA = VO(vbus=36 * unit.V, iload=50 * unit.mA, tag="VOUT_50mA")
IUNC_50mA = IUNC(vbus=36 * unit.V, iload=50 * unit.mA, tag="IUNC_50mA")

print([VOUT_50mA, IUNC_50mA])

[VOUT_50mA: 252.2 mV (nom), -1.193 V (lb), 1.642 V (ub),
 IUNC_50mA: 0 mA (nom), -286.5 mA (lb), 275.5 mA (ub)]


In [8]:
# perform sensitivity study at the 36V, 50mA operating point
IUNC_50mA_sensitivities = [
    IUNC_50mA(tag="IUNC_50mA-R1-sens").ss(R1),
    IUNC_50mA(tag="IUNC_50mA-VOS-sens").ss(VOS),
    IUNC_50mA(tag="IUNC_50mA-R2-thru-R5-sens").ss([R2, R3, R4, R5]),
]

print(IUNC_50mA_sensitivities)

[IUNC_50mA-R1-sens: 0 mA (nom), -500 µA (lb), 500 µA (ub),
 IUNC_50mA-VOS-sens: 0 mA (nom), -1.53 mA (lb), 1.53 mA (ub),
 IUNC_50mA-R2-thru-R5-sens: 0 mA (nom), -284.4 mA (lb), 273.5 mA (ub)]


## Numerically Solve for the Common-Mode Gain

In [9]:
@derive.byev(R1, R2, R3, R4, R5, VOS, tag="CMGAIN")
def CMGAIN(r1, r2, r3, r4, r5, vos):
    def cmgain_constraints(vbus, iload):
        vo = VO(vbus, iload, r1, r2, r3, r4, r5, vos)
        vdm = r1 * iload
        vcm = vbus - r1 * iload / 2
        return vo.m, vdm.m, vcm.m  # strip units for scipy compatibility

    # three equations for three unknowns (Adm, Acm, Voff)
    vo1, vdm1, vcm1 = cmgain_constraints(0 * unit.V, 0 * unit.A)
    vo2, vdm2, vcm2 = cmgain_constraints(1 * unit.V, 10 * unit.A)
    vo3, vdm3, vcm3 = cmgain_constraints(3 * unit.V, 20 * unit.A)

    b = np.array([vo1, vo2, vo3])
    A = np.array([[vcm1, vdm1, 1], [vcm2, vdm2, 1], [vcm3, vdm3, 1]])
    x = solve(A, b)
    return x[0] * unit([])  # reappend units (dimensionless)

In [10]:
print(CMGAIN)

CMGAIN: 2.842E-14 (nom), -0.04 (lb), 0.03846 (ub)


In [11]:
CMGAIN_sensitivity = CMGAIN(tag="CMGAIN-R2-thru-R5-sens").ss([R2, R3, R4, R5])
print(CMGAIN_sensitivity)

CMGAIN-R2-thru-R5-sens: 2.842E-14 (nom), -0.04 (lb), 0.03846 (ub)
